In [1]:
from tqdm import tqdm
from cobra import Reaction
import escher
import escher.urls
import cobra
import cobra.test
import json
import os
import string
from IPython.display import HTML
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.parsimonious import optimize_minimal_flux
from cobra import ArrayBasedModel
import numpy as np
model_ref = cobra.io.load_matlab_model('C:\\Users\\Omar\\Desktop\\virtual\\venv\\RECON1', 'RECON1')

# 1) Get list of lists of unique genes and corresponding reactions

In [2]:
gprs = []
rxns = []
# get reacions in model and the associate gprs
for reaction in model_ref.reactions:
    rxns.append(reaction)
    gpr = reaction.gene_reaction_rule
    gprs.append(gpr)

In [3]:
with open('mann_biserial_cutoff_down_single.csv','r') as bis:
    clus_list = []
    ent_list = []
    genes_names_list = []
    biserial_list=[]
    log2fold_list = []
    for line in bis:
        line_ele = line.strip().split(",")
        clus_list.append(line_ele[0])
        ent_list.append(line_ele[1])
        genes_names_list.append(line_ele[2])
        biserial_list.append(line_ele[3])
        log2fold_list.append(line_ele[4])

In [4]:
unique_ent = []
for i in ent_list:
    if i not in unique_ent:
        unique_ent.append(i)

In [5]:
#rxns_dict={}
count = 0
overall_rxns_incl_down =[]
for i,x in enumerate(unique_ent):
    rxns_incl = []
    gpr_incl = []
    for reaction in model_ref.reactions:
        gpr = reaction.gene_reaction_rule
        gpr = gpr.replace(')','')
        gpr = gpr.replace('(','')
        gpr = gpr.strip().split()
        
        for item in gpr:
            if x + '_AT1' in item or x+'_AT2' in item or x + '_AT3' in item or x+'_AT4' in item or x+'_AT5' in item:
                if len(item) == len(x + '_AT1'):
                    rxns_incl.append(str(reaction))
                    gpr_incl.append(gpr)
                    #print ent,reaction, gpr
                    count +=1
    overall_rxns_incl_down.append(rxns_incl)
    #rxns_dict[ent] = rxns_incl

In [6]:
unique_rxns = []
for i in range(len(overall_rxns_incl_down)):
    cur_set = overall_rxns_incl_down[i]
    cur_uniques=[]
    for j in range(len(cur_set)):
        if cur_set[j] not in cur_uniques:
            cur_uniques.append(cur_set[j])
    unique_rxns.append(cur_uniques)

# 2) Set up biomass reaction as well as RECON1 -- note: EX__arg is set at a lb of -0.22

## this is done such that a flux solution can be found in the healthy case

* create demands for: ethanalamine (lb set at -1), 
* create a transporter for Tyr_ggnt (lb set at -1000)
* change all DM_ and EX_ lower bounds to 0
* set the lower bounds to -1 of the following exceptions: EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e
* set EX_yvite_e and EX_10fthf5glu_e to 0 as well
* add the biomass reaction as per the following
* add the biomass reaction to the model and set as model objective
* add a DM_atp_c demand DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1}) with the lower bound set as 0.9*7.9939
* free the following reactions model.reactions.get_by_id('EX_h_e').lower_bound = -1000
model.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model.reactions.get_by_id('EX_nh4_e').lower_bound = -1000
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4882
model.reactions.get_by_id('EX_pi_e').lower_bound = -1000

* constrain the upper and lower bound of the biomass reaction to 0.0177, obtained from calculate fluxes

In [7]:
#creat ethanalamine demand
#metabolite is etha_c

DM_etha_c = Reaction('DM_etha_c')
DM_etha_c.name = DM_etha_c

etha_c = model_ref.metabolites.get_by_id('etha_c')

DM_etha_c.add_metabolites({etha_c: -1})
model_ref.add_reaction(DM_etha_c)

#create a transporter for this

Tyr_ggnt = Reaction('Tyr_ggnt')
Tyr_ggnt.name = Tyr_ggnt

Tyr_ggn_e = model_ref.metabolites.get_by_id('Tyr_ggn_e')
Tyr_ggn_c = model_ref.metabolites.get_by_id('Tyr_ggn_c')

Tyr_ggnt.add_metabolites({Tyr_ggn_e: -1, Tyr_ggn_c: 1})
model_ref.add_reaction(Tyr_ggnt)

DM_etha_c.lower_bound = -1
Tyr_ggnt.lower_bound = -1000

except_EX_names = '''DM_etha_c, EX_peplys_e, EX_Tyr_ggn_e, EX_arg__L_e, EX_asn__L_e, EX_asp__L_e, EX_chol_e, EX_cl_e, EX_glc__D_e, EX_gln__L_e, EX_gly_e, EX_h_e, EX_h2o_e, EX_ile__L_e, EX_k_e, EX_leu__L_e, EX_lys__L_e, EX_na1_e, EX_nh4_e, EX_o2_e, EX_phe__L_e, EX_pi_e, EX_pro__L_e, EX_ser__L_e, EX_thr__L_e, EX_trp__L_e, EX_tyr__L_e, EX_val__L_e'''
except_EX_names_split = except_EX_names.split(', ')
print except_EX_names_split

# Changing all lower bounds to 0 except the given reactions above

for a in model_ref.reactions[1188:1591]:
    a.lower_bound = 0

model_ref.reactions.get_by_id('EX_yvite_e').lower_bound = 0
model_ref.reactions.get_by_id('EX_10fthf5glu_e').lower_bound = 0
    
for b in model_ref.reactions[1188:1591]:
    for c in range(len(except_EX_names_split)):  
        if b.id  == except_EX_names_split[c]:
            b.lower_bound = -1
            
biomass_NCI60 = Reaction('biomass_NCI60')
biomass_NCI60.name = biomass_NCI60

ala_L_c = model_ref.metabolites.get_by_id('ala__L_c')
arg_L_c = model_ref.metabolites.get_by_id('arg__L_c')
asn_L_c = model_ref.metabolites.get_by_id('asn__L_c')
asp_L_c = model_ref.metabolites.get_by_id('asp__L_c')
atp_c = model_ref.metabolites.get_by_id('atp_c')
clpn_hs_c = model_ref.metabolites.get_by_id('clpn_hs_c')
ctp_c = model_ref.metabolites.get_by_id('ctp_c')
dag_hs_c = model_ref.metabolites.get_by_id('dag_hs_c')
datp_c = model_ref.metabolites.get_by_id('datp_c')
dctp_c = model_ref.metabolites.get_by_id('dctp_c')
dgtp_c = model_ref.metabolites.get_by_id('dgtp_c')
dttp_c = model_ref.metabolites.get_by_id('dttp_c')
gln_L_c = model_ref.metabolites.get_by_id('gln__L_c')
glu_L_c = model_ref.metabolites.get_by_id('glu__L_c')
gly_c = model_ref.metabolites.get_by_id('gly_c')
glygn2_c = model_ref.metabolites.get_by_id('glygn2_c')
gtp_c = model_ref.metabolites.get_by_id('gtp_c')
h2o_c = model_ref.metabolites.get_by_id('h2o_c')
hdca_c = model_ref.metabolites.get_by_id('hdca_c')
hdcea_c = model_ref.metabolites.get_by_id('hdcea_c')
ile_L_c = model_ref.metabolites.get_by_id('ile__L_c')
leu_L_c = model_ref.metabolites.get_by_id('leu__L_c')
lpchol_hs_c = model_ref.metabolites.get_by_id('lpchol_hs_c')
lys_L_c = model_ref.metabolites.get_by_id('lys__L_c')
mag_hs_c = model_ref.metabolites.get_by_id('mag_hs_c')
ocdca_c = model_ref.metabolites.get_by_id('ocdca_c')
ocdcea_c = model_ref.metabolites.get_by_id('ocdcea_c')
pa_hs_c = model_ref.metabolites.get_by_id('pa_hs_c')
pail_hs_c = model_ref.metabolites.get_by_id('pail_hs_c')
pchol_hs_c = model_ref.metabolites.get_by_id('pchol_hs_c')
pe_hs_c = model_ref.metabolites.get_by_id('pe_hs_c')
phe_L_c = model_ref.metabolites.get_by_id('phe__L_c')
ps_hs_c = model_ref.metabolites.get_by_id('ps_hs_c')
ser_L_c = model_ref.metabolites.get_by_id('ser__L_c')
sphmyln_hs_c = model_ref.metabolites.get_by_id('sphmyln_hs_c')
thr_L_c = model_ref.metabolites.get_by_id('thr__L_c')
trp_L_c = model_ref.metabolites.get_by_id('trp__L_c')
tyr_L_c = model_ref.metabolites.get_by_id('tyr__L_c')
utp_c = model_ref.metabolites.get_by_id('utp_c')
val_L_c = model_ref.metabolites.get_by_id('val__L_c')
pro_L_m = model_ref.metabolites.get_by_id('pro__L_m')
chsterol_r = model_ref.metabolites.get_by_id('chsterol_r')
xolest_hs_r = model_ref.metabolites.get_by_id('xolest_hs_r')
adp_c = model_ref.metabolites.get_by_id('adp_c')
h_c = model_ref.metabolites.get_by_id('h_c')
pi_c = model_ref.metabolites.get_by_id('pi_c')

#remove ocdca and ocdcea
# added xolest value to cholesterol in biomass and remove xolest

biomass_NCI60.add_metabolites({ala_L_c: -0.587929, arg_L_c: -0.380280, asn_L_c: -0.323313, asp_L_c: -0.261396, 
                               atp_c: -35.033540, clpn_hs_c: -0.000624, ctp_c: -0.033435, dag_hs_c: -0.001032, 
                               datp_c: -0.014557, dctp_c: -0.009770, dgtp_c: -0.009748, dttp_c: -0.014546,
                               gln_L_c: -0.319051, glu_L_c: -0.387401, gly_c: -0.504294, glygn2_c: -0.034479,
                               gtp_c: -0.055967, h2o_c: -35.000000, hdca_c: -0.008293, hdcea_c: -0.003315, 
                               ile_L_c: -0.319813, leu_L_c: -0.548692, lpchol_hs_c: -0.002470, lys_L_c: -0.552717, 
                               mag_hs_c: -0.001456, pa_hs_c: -0.010645,
                               pail_hs_c: -0.005016, pchol_hs_c: -0.022878, pe_hs_c: -0.018211, phe_L_c: -0.170743,
                               ps_hs_c: -0.006808, ser_L_c: -0.385852, sphmyln_hs_c: -0.010215, thr_L_c: -0.378004, 
                               trp_L_c: -0.039847, tyr_L_c: -0.150141, utp_c: -0.063323, val_L_c: -0.385554,
                               pro_L_m: -0.237850, chsterol_r: -0.054102,  adp_c: 35.000000, 
                               h_c: 35.000000, pi_c: 35.000000})

model_ref.add_reaction(biomass_NCI60)
my_objective = model_ref.reactions.get_by_id('biomass_NCI60')
model_ref.change_objective(my_objective)

# Adding DM_atp_c reaction

DM_atp_c = Reaction('DM_atp_c')
DM_atp_c.name = DM_atp_c

atp_c = model_ref.metabolites.get_by_id('atp_c')
adp_c = model_ref.metabolites.get_by_id('adp_c')
h2o_c = model_ref.metabolites.get_by_id('h2o_c')
h_c = model_ref.metabolites.get_by_id('h_c')
pi_c = model_ref.metabolites.get_by_id('pi_c')


DM_atp_c.add_metabolites({atp_c: -1, h2o_c: -1, adp_c: 1, pi_c: 1, h_c: 1})
model_ref.add_reaction(DM_atp_c)

# Setting lower bounds for DM_atp_c as per Dan

model_ref.reactions.get_by_id('DM_atp_c').lower_bound = 0.9*7.9939

model_ref.reactions.get_by_id('EX_h_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_o2_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_fe2_e').lower_bound = -1000
model_ref.reactions.get_by_id('EX_nh4_e').lower_bound = -1000

model_ref.reactions.get_by_id('EX_glc__D_e').lower_bound = -0.4882

# constrained as per calculate fluxes 
model_ref.reactions.get_by_id('biomass_NCI60').lower_bound = 0.0177
model_ref.reactions.get_by_id('biomass_NCI60').upper_bound = 0.0177

model_ref.reactions.get_by_id('EX_pi_e').lower_bound = -1000

# 6/16/16 Changing amino acid uptake constraints
# arg__L__e initially was -0.0022
model_ref.reactions.get_by_id('EX_arg__L_e').lower_bound = -0.0022

model_ref.reactions.get_by_id('EX_asn__L_e').lower_bound = -0.0058
model_ref.reactions.get_by_id('EX_asp__L_e').lower_bound = -0.0081
model_ref.reactions.get_by_id('EX_chol_e').lower_bound = -0.00062386
model_ref.reactions.get_by_id('EX_gln__L_e').lower_bound = -0.1202
model_ref.reactions.get_by_id('EX_gly_e').lower_bound = -0.0019
model_ref.reactions.get_by_id('EX_ile__L_e').lower_bound = -0.0081
model_ref.reactions.get_by_id('EX_leu__L_e').lower_bound = -0.0110
model_ref.reactions.get_by_id('EX_lys__L_e').lower_bound = -0.0111
model_ref.reactions.get_by_id('EX_phe__L_e').lower_bound = -0.0040

model_ref.reactions.get_by_id('EX_pro__L_e').lower_bound = -0.0045

model_ref.reactions.get_by_id('EX_ser__L_e').lower_bound = -0.0167
model_ref.reactions.get_by_id('EX_trp__L_e').lower_bound = -0.0013
model_ref.reactions.get_by_id('EX_tyr__L_e').lower_bound = -0.0068
model_ref.reactions.get_by_id('EX_val__L_e').lower_bound = -0.0079

#6/24/16 lowerbound changes

model_ref.reactions.get_by_id('FTHFLi').lower_bound = 0
model_ref.reactions.get_by_id('FTHFLmi').lower_bound = 0

model_ref.reactions.get_by_id('FUM').lower_bound = 0
model_ref.reactions.get_by_id('FUM').upper_bound = 0

model_ref.reactions.get_by_id('BILGLCURte').lower_bound = 0
model_ref.reactions.get_by_id('BILDGLCURte').lower_bound = 0

h_i = Metabolite('h_i', formula='h', name='h_i', compartment='i')
model_ref.add_metabolites(h_i)


# delete reaction
model_ref.remove_reactions('ATPS4m')

# re-add the reaction while replacing h_c with h_i
ATPS4m = Reaction('ATPS4m')
ATPS4m.name = ATPS4m

adp_m = model_ref.metabolites.get_by_id('adp_m')
h_i = model_ref.metabolites.get_by_id('h_i')
pi_m = model_ref.metabolites.get_by_id('pi_m')
h2o_m = model_ref.metabolites.get_by_id('h2o_m')
h_m = model_ref.metabolites.get_by_id('h_m')
atp_m = model_ref.metabolites.get_by_id('atp_m')

ATPS4m.add_metabolites({adp_m: -1, h_i: -4, pi_m: -1, h2o_m: 1, h_m: 3, atp_m: 1})
model_ref.add_reaction(ATPS4m)

model_ref.remove_reactions('CYOOm3')

CYOOm3 = Reaction('CYOOm3')
CYOOm3.name = CYOOm3

h_m = model_ref.metabolites.get_by_id('h_m')
o2_m = model_ref.metabolites.get_by_id('o2_m')
focytC_m = model_ref.metabolites.get_by_id('focytC_m')
h_i = model_ref.metabolites.get_by_id('h_i')
h2o_m = model_ref.metabolites.get_by_id('h2o_m')
ficytC_m = model_ref.metabolites.get_by_id('ficytC_m')


CYOOm3.add_metabolites({h_m: -8, o2_m: -1, focytC_m: -4, h_i: 4, h2o_m: 2, ficytC_m: 4})
model_ref.add_reaction(CYOOm3)

# delete reaction
model_ref.remove_reactions('NADH2_u10m')

# re-add the reaction while replacing h_c with h_i
NADH2_u10m = Reaction('NADH2_u10m')
NADH2_u10m.name = NADH2_u10m

q10_m = model_ref.metabolites.get_by_id('q10_m')
h_m = model_ref.metabolites.get_by_id('h_m')
nadh_m = model_ref.metabolites.get_by_id('nadh_m')
nad_m = model_ref.metabolites.get_by_id('nad_m')
h_i = model_ref.metabolites.get_by_id('h_i')
q10h2_m = model_ref.metabolites.get_by_id('q10h2_m')



NADH2_u10m.add_metabolites({q10_m: -1, h_m: -5, nadh_m: -1, nad_m: 1, h_i: 4, q10h2_m: 1})
model_ref.add_reaction(NADH2_u10m)

# delete reaction
model_ref.remove_reactions('CYOR_u10m')

# re-add the reaction while replacing h_c with h_i
CYOR_u10m = Reaction('CYOR_u10m')
CYOR_u10m.name = NADH2_u10m

ficytC_m = model_ref.metabolites.get_by_id('ficytC_m')
h_m = model_ref.metabolites.get_by_id('h_m')
q10h2_m = model_ref.metabolites.get_by_id('q10h2_m')
q10_m = model_ref.metabolites.get_by_id('q10_m')
focytC_m = model_ref.metabolites.get_by_id('focytC_m')
h_i = model_ref.metabolites.get_by_id('h_i')


CYOR_u10m.add_metabolites({ficytC_m: -2, h_m: -2, q10h2_m: -1, q10_m: 1, focytC_m: 2, h_i: 4})
model_ref.add_reaction(CYOR_u10m)

['DM_etha_c', 'EX_peplys_e', 'EX_Tyr_ggn_e', 'EX_arg__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_chol_e', 'EX_cl_e', 'EX_glc__D_e', 'EX_gln__L_e', 'EX_gly_e', 'EX_h_e', 'EX_h2o_e', 'EX_ile__L_e', 'EX_k_e', 'EX_leu__L_e', 'EX_lys__L_e', 'EX_na1_e', 'EX_nh4_e', 'EX_o2_e', 'EX_phe__L_e', 'EX_pi_e', 'EX_pro__L_e', 'EX_ser__L_e', 'EX_thr__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_val__L_e']


cobra\core\Model.py:289 UserWarning: need to pass in a list


### Find loops and reduce model

In [8]:
model_test = model_ref.copy()

In [9]:
# allow 0 to be and option for FVA
for rxn in model_test.reactions:
    if model_test.reactions.get_by_id(str(rxn)).lower_bound > 0:
        model_test.reactions.get_by_id(str(rxn)).lower_bound = 0
    elif model_test.reactions.get_by_id(str(rxn)).upper_bound < 0:
        model_test.reactions.get_by_id(str(rxn)).upper_bound = 0

In [10]:
model_test.optimize()

<Solution 0.02 at 0x10bd8978>

In [11]:
fva_result = cobra.flux_analysis.flux_variability_analysis(model_test)

In [12]:
#create reduced model based on FVA
# not we should be doing this after adding each demand since we can activate pathways 
model_red = model_ref.copy()
for key in fva_result:
    if -10**-6< fva_result[key]['maximum'] < 10**-6 and -10**-6< fva_result[key]['minimum'] < 10**-6:
        model_red.reactions.get_by_id(key).delete(remove_orphans = True)

In [13]:
model_healthy_red = model_red.copy()

# 3) Run pFBA with biomass set as the objective

## Calculating the normal, healthy demands

In [14]:
p_solution_ref = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model_ref)

In [15]:
p_sol = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model_healthy_red)

In [16]:
p_solution_ref.x_dict['SUCD1m']

0.4979664887656241

In [17]:
p_sol.x_dict['SUCD1m']

0.4979664887656219

#### Constrain reactions 

In [18]:
from cobra.flux_analysis.loopless import construct_loopless_model

In [42]:
unique_rxns = [['SUCD1m'],['ARGSS'],['FUMm']]

In [19]:
unique_rxns = [['FUMm']]

## Original Loopless

In [20]:
run ={}
final ={}

# essentially for each gene to be knocked out
for i1 in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_healthy_red.copy()

    
    ###### CREATE KO MODEL ######
    model_KO = model.copy()
    for reaction in unique_rxns[i1]:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0
        
    ###### SHADOW PRICE ANALYSIS ######
    model_sp = model_KO.copy()
    model_sp.reactions.get_by_id('biomass_NCI60').lower_bound = 0
    model_sp.optimize(solver='gurobi').f
    # get shadow price values for each metabolite
    met_sp = []
    try:
        for met in model_sp.metabolites:
            met_sp.append(met.y)
    except Exception:
        model_sp.change_objective('DM_atp_c')
        model_sp.optimize(solver='gurobi').f
        for met in model_sp.metabolites:
            met_sp.append(met.y)
        print unique_rxns[i]
    # setup a zero cutoff
    for ind,item in enumerate(met_sp):
        if -10**-6<item<10**-6:
            met_sp[ind] = 0
    # get list of biomass metabolites
    biomass_mets = []
    for met in model_sp.reactions.get_by_id('biomass_NCI60').metabolites:
        biomass_mets.append(str(met))

    # get a list of the non-zero shadow prices and cross-reference biomass metabolites to find limiting nutrient(s) if any
    possible_limiting_nutrients = []
    limiting_nutrients = []
    for ind,item in enumerate(met_sp):
        if item !=0:
            possible_limiting_nutrients.append(str(model_sp.metabolites[ind]))
            possible = str(model_sp.metabolites[ind])
            if possible in biomass_mets:
                limiting_nutrients.append(possible)
    # get a list of new objective reactions which should be the corresponding exchange reaction(s) if any
    if limiting_nutrients:
        new_objective_reactions = []
        for item in limiting_nutrients:
            item = item.replace(item[-1],'e')
            item = 'EX_' + item
            new_objective_reactions.append(item)
        #run[unique_ent[i]] = new_objective_reactions   
        #### DETERMINING NEW UPTAKE ####
        for n in range(len(new_objective_reactions)):
            model_new_up = model_KO.copy()
            model_new_up.objective = model.reactions.get_by_id(new_objective_reactions[n])
            model_new_up.reactions.get_by_id(new_objective_reactions[n]).lower_bound = -1000
            new_up = model_new_up.optimize(solver='gurobi').f
            model.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up
            model_KO.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up

    
    for reaction in unique_rxns[i1]:
        print reaction
        
        model.reactions.get_by_id(reaction).lower_bound = p_sol.x_dict[reaction]


        newmet_array = [0] * len(model.metabolites)
        met_count = 0
        for i in range(len(model.metabolites)):
            newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
            met_count = met_count + 1

        #####ALGORITHM################
        # creates an empty dict to store DM_rxns as keys and the p_solution.f values
        normal_maxmet_dict = {}

        for i in range(len(model.metabolites)):
            #for i in range(80):

            #  switch toggle
            switch = True

            # this part creates a demand reaction for each metabolite
            temp_DM = 'DM_' + str(newmet_array[i])

            # this nested for loop tries to catch if a demand reaction already exists in the model
            for ii in model.reactions:
                if temp_DM == str(ii):
                    # this part sets the newly created demand as the objective
                    my_objective = model.reactions.get_by_id(str(temp_DM))
                    model.change_objective(my_objective)

                    # this part runs FBA on the new demand
                    p_solution = model.optimize(solver='gurobi')

                    # check if p_solution is above a tolerance to check if we want to run loopless or not
                    if p_solution < 10**-6:
                        normal_maxmet_dict[str(temp_DM)] = p_solution.f

                    else:
                        solution = construct_loopless_model(model).optimize(solver='gurobi')
                        normal_maxmet_dict[str(temp_DM)] = solution.f

                    # a toggle to see if the reaction is to be deleted or not
                    switch = False

            if switch is True:
                temp_DM = Reaction(str(temp_DM))
                newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
                temp_DM.add_metabolites({newmet_array[i]: -1})
                model.add_reaction(temp_DM)

                # this part sets the newly created demand as the objective
                my_objective = model.reactions.get_by_id(str(temp_DM))
                model.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model.optimize(solver='gurobi')

                # check if p_solution is above a tolerance to check if we want to run loopless or not
                if p_solution < 10**-6:
                    normal_maxmet_dict[str(temp_DM)] = p_solution.f

                else:
                    solution = construct_loopless_model(model).optimize(solver='gurobi')
                    normal_maxmet_dict[str(temp_DM)] = solution.f

                # this deletes the demand at the end of each iteration
                model.reactions.get_by_id(str(temp_DM)).delete()
                
                
        # working copy!
        # This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code
        newmet_array = [0] * len(model_KO.metabolites)
        met_count = 0
        for i in range(len(model_KO.metabolites)):
            newmet_array[met_count] = string.replace(model_KO.metabolites[i].id, " ", "_")
            met_count = met_count + 1


        # creates an empty dict to store DM_rxns as keys and the p_solution.f values
        KO_maxmet_dict = {}

        for i in range(len(model_KO.metabolites)):
        #for i in range(80):

            # switch toggle
            switch = True

            # this part creates a demand reaction for each metabolite
            temp_DM = 'DM_' + str(newmet_array[i])

            # this nested for loop tries to catch if a demand reaction already exists in the model
            for ii in model_KO.reactions:
                if temp_DM == str(ii):
                    # this part sets the newly created demand as the objective
                    my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                    model_KO.change_objective(my_objective)

                    # this part runs FBA on the new demand
                    p_solution = model_KO.optimize(solver='gurobi')

                    # this stores the demand flux into a dictionary where its demand rxn is the key
                    KO_maxmet_dict[str(temp_DM)] = p_solution.f

                    # a toggle to see if the reaction is to be deleted or not
                    switch = False

            if switch is True:
                temp_DM = Reaction(str(temp_DM))
                newmet_array[i] = model_KO.metabolites.get_by_id(model_KO.metabolites[i].id)
                temp_DM.add_metabolites({newmet_array[i]: -1})
                model_KO.add_reaction(temp_DM)

                # this part sets the newly created demand as the objective
                my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                model_KO.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model_KO.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                KO_maxmet_dict[str(temp_DM)] = p_solution.f

                # this deletes the demand at the end of each iteration
                model_KO.reactions.get_by_id(str(temp_DM)).delete()
                
        run[str(reaction) + '_normal'] = normal_maxmet_dict
        run[str(reaction) + '_KO'] = KO_maxmet_dict
        
    final[str(i1)] = run


FUMm


In [22]:
run

{'FUMm_KO': {'DM_vitd3_e': 0.033815717149024274,
  'DM_3mox4hoxm_c': -0.0,
  'DM_malcoa_m': 0.23277655514418982,
  'DM_4fumacac_c': 0.005120353199999999,
  'DM_vitd3_c': 0.03381571710440577,
  'DM_dttp_m': 0.14016510144574568,
  'DM_vitd3_m': 0.033815716985765604,
  'DM_1pyr5c_c': 0.3289058057125658,
  'DM_malcoa_c': 0.22940298603006903,
  'DM_R1coa_hs_c': 0.06340446966161283,
  'DM_xoldioloneh_c': 0.032758976000626196,
  'DM_vacccrn_c': -0.0,
  'DM_mepi_e': 0.005120353199999999,
  'DM_pail4p_hs_n': 0.027251834307460948,
  'DM_mepi_c': 0.005120353199999999,
  'DM_1p3h5c_m': -0.0,
  'DM_1pyr5c_m': 0.3289058100257309,
  'DM_Tyr_ggn_e': 0.9993897217,
  'DM_adprib_c': 0.07397602605179925,
  'DM_hmbil_c': 0.044184521641000174,
  'DM_f1p_c': 0.2467323246893341,
  'DM_clpn_hs_c': 0.014792387848611461,
  'DM_Tyr_ggn_c': 0.9993897217,
  'DM_hdcoa_m': 0.06215141676024073,
  'DM_thcholstoic_m': 0.03209042534490671,
  'DM_3mob_c': 0.0010756942000000004,
  'DM_q10_m': -0.0,
  'DM_pail34p_hs_n': 0.0

In [24]:
out = open('loopless_FH_normal.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = normal_maxmet_dict[temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

In [25]:
out = open('loopless_FH_KO.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = KO_maxmet_dict[temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

In [95]:
for i in range(len(newmet_array)):
    if str(newmet_array[i]) == str(model.metabolites.get_by_id('ala_B_e')):
        print i

441


In [94]:
newmet_array[551]

<Metabolite for_e at 0x19897a20>

In [ ]:
newmet_array

In [92]:
model.metabolites.get_by_id('for_e')

<Metabolite for_e at 0x14fd3780>

In [70]:
model.objective = model.reactions.get_by_id('EX_bilglcur_e')

In [71]:
model.optimize().f

0.03817247931146994

In [72]:
model.optimize().x_dict['EX_bilglcur_e']

0.03817247931146994

In [73]:
model.reactions.get_by_id('HMBS')

<Reaction HMBS at 0x215f7f28>

In [82]:
model = model_healthy_red.copy()

In [83]:
temp_DM = Reaction('DM_bilglcur_e')
temp_met = model.metabolites.get_by_id('bilglcur_e')
temp_DM.add_metabolites({temp_met: -1})
model.add_reaction(temp_DM)

In [84]:
model.objective = model.reactions.get_by_id('DM_bilglcur_e')

In [88]:
check_sol = model.optimize()

In [89]:
check_sol.f

0.03817247931146994

In [85]:
solution = construct_loopless_model(model).optimize(solver='gurobi')

In [90]:
solution

<Solution 'infeasible' at 0x216680f0>

In [ ]:
model.reactions.get_by_id(str(temp_DM)).delete()

## DOING KO Only FH

In [29]:
run ={}
final ={}

# essentially for each gene to be knocked out
for i1 in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_healthy_red.copy()

    
    ###### CREATE KO MODEL ######
    model_KO = model.copy()
    for reaction in unique_rxns[i1]:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0
        
    ###### SHADOW PRICE ANALYSIS ######
    model_sp = model_KO.copy()
    model_sp.reactions.get_by_id('biomass_NCI60').lower_bound = 0
    model_sp.optimize(solver='gurobi').f
    # get shadow price values for each metabolite
    met_sp = []
    try:
        for met in model_sp.metabolites:
            met_sp.append(met.y)
    except Exception:
        model_sp.change_objective('DM_atp_c')
        model_sp.optimize(solver='gurobi').f
        for met in model_sp.metabolites:
            met_sp.append(met.y)
        print unique_rxns[i]
    # setup a zero cutoff
    for ind,item in enumerate(met_sp):
        if -10**-6<item<10**-6:
            met_sp[ind] = 0
    # get list of biomass metabolites
    biomass_mets = []
    for met in model_sp.reactions.get_by_id('biomass_NCI60').metabolites:
        biomass_mets.append(str(met))

    # get a list of the non-zero shadow prices and cross-reference biomass metabolites to find limiting nutrient(s) if any
    possible_limiting_nutrients = []
    limiting_nutrients = []
    for ind,item in enumerate(met_sp):
        if item !=0:
            possible_limiting_nutrients.append(str(model_sp.metabolites[ind]))
            possible = str(model_sp.metabolites[ind])
            if possible in biomass_mets:
                limiting_nutrients.append(possible)
    # get a list of new objective reactions which should be the corresponding exchange reaction(s) if any
    if limiting_nutrients:
        new_objective_reactions = []
        for item in limiting_nutrients:
            item = item.replace(item[-1],'e')
            item = 'EX_' + item
            new_objective_reactions.append(item)
        #run[unique_ent[i]] = new_objective_reactions   
        #### DETERMINING NEW UPTAKE ####
        for n in range(len(new_objective_reactions)):
            model_new_up = model_KO.copy()
            model_new_up.objective = model.reactions.get_by_id(new_objective_reactions[n])
            model_new_up.reactions.get_by_id(new_objective_reactions[n]).lower_bound = -1000
            new_up = model_new_up.optimize(solver='gurobi').f
            model.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up
            model_KO.reactions.get_by_id(new_objective_reactions[n]).lower_bound = new_up

    
    for reaction in unique_rxns[i1]:
        print reaction
        
        model.reactions.get_by_id(reaction).lower_bound = p_sol.x_dict[reaction]
                
        # working copy!
        # This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code
        newmet_array = [0] * len(model_KO.metabolites)
        met_count = 0
        for i in range(len(model_KO.metabolites)):
            newmet_array[met_count] = string.replace(model_KO.metabolites[i].id, " ", "_")
            met_count = met_count + 1


        # creates an empty dict to store DM_rxns as keys and the p_solution.f values
        KO_maxmet_dict = {}

        for i in range(len(model_KO.metabolites)):
        #for i in range(80):

            # switch toggle
            switch = True

            # this part creates a demand reaction for each metabolite
            temp_DM = 'DM_' + str(newmet_array[i])

            # this nested for loop tries to catch if a demand reaction already exists in the model
            for ii in model_KO.reactions:
                if temp_DM == str(ii):
                    # this part sets the newly created demand as the objective
                    my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                    model_KO.change_objective(my_objective)

                    # this part runs FBA on the new demand
                    p_solution = model_KO.optimize(solver='gurobi')

                    # this stores the demand flux into a dictionary where its demand rxn is the key
                    KO_maxmet_dict[str(temp_DM)] = p_solution.f

                    # a toggle to see if the reaction is to be deleted or not
                    switch = False

            if switch is True:
                temp_DM = Reaction(str(temp_DM))
                newmet_array[i] = model_KO.metabolites.get_by_id(model_KO.metabolites[i].id)
                temp_DM.add_metabolites({newmet_array[i]: -1})
                model_KO.add_reaction(temp_DM)

                # this part sets the newly created demand as the objective
                my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                model_KO.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model_KO.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                KO_maxmet_dict[str(temp_DM)] = p_solution.f

                # this deletes the demand at the end of each iteration
                model_KO.reactions.get_by_id(str(temp_DM)).delete()
                
        #run[str(reaction) + '_normal'] = normal_maxmet_dict
        run[str(reaction) + '_KO'] = KO_maxmet_dict
        
    final[str(i1)] = run


FUMm


In [30]:
out = open('new_check_FH_KO.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = KO_maxmet_dict[temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

In [74]:
KO_maxmet_dict['DM_thym_c']

0.34701526729068743

In [75]:
normal_maxmet_dict['DM_thym_c']

0.3453964942891095

In [67]:
out = open('loopless_ARGSS_normal.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = normal_maxmet_dict[temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

In [68]:
out = open('loopless_ARGSS_KO.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = KO_maxmet_dict[temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

## Computing and normalizing the demands to generate metabolite scores

In [76]:
# This combines the normal_flux solutions and KO_flux solutions into one dictionary
# The same key, the reaction, and stores 2 flux values as a list to the dic key where [0] is normal and [1] is KO

combined_dic = {key: {'init_flux': normal_maxmet_dict[key], 'KO_flux': KO_maxmet_dict[key]}
                for key in KO_maxmet_dict if key in normal_maxmet_dict}

In [78]:
combined_dic['DM_thym_c']

{'KO_flux': 0.34701526729068743, 'init_flux': 0.3453964942891095}

In [80]:
# Setting the cutoff to 10^-5

for rxn, flux in combined_dic.iteritems():
    # 10^-5
    if round(flux['init_flux'], 6) < 0.00001:
        flux['init_flux'] = 0
    if round(flux['KO_flux'], 6) < 0.00001:
        flux['KO_flux'] = 0

TypeError: a float is required

In [86]:
KO_maxmet_dict['DM_for_e']

1.3911240248920278

In [51]:
out = open('loopless_ARGSS_1.csv','w')
for met in model.metabolites:
    temp_DM = 'DM_' + str(met)
    val = run['ARGSS_normal'][temp_DM]
    out.write('%s,%s\n'%(temp_DM,val))
out.close()

In [52]:
run['ARGSS_normal']['DM_adprib_c']

0.11648134212366407

In [34]:
model_KO.reactions.get_by_id('EX_arg__L_e').lower_bound = -0.0068

In [35]:
model_KO.objective = model.reactions.get_by_id('biomass_NCI60')

In [36]:
model_KO.optimize().f

0.0177

In [38]:
temp_DM = Reaction('DM_adprib_c')
met = model_KO.metabolites.get_by_id('adprib_c')
temp_DM.add_metabolites({met: -1})
model_KO.add_reaction(temp_DM)

# this part sets the newly created demand as the objective
my_objective = model_KO.reactions.get_by_id(str(temp_DM))
model_KO.change_objective(my_objective)

# this part runs FBA on the new demand
p_solution = model_KO.optimize(solver='gurobi')

In [40]:
p_solution.f

0.12025338109873418

In [68]:
unique_rxns = [['ARGSS']]

In [84]:
for i1 in range(len(unique_rxns)):
    # create a copy of reference model so don't have to worry about things accidently adding up
    model = model_healthy_red.copy()
    
    ###### CREATE KO MODEL ######
    model_KO = model.copy()
    for reaction in unique_rxns[i1]:
        model_KO.reactions.get_by_id(reaction).lower_bound = 0
        model_KO.reactions.get_by_id(reaction).upper_bound = 0
    
    for reaction in unique_rxns[i1]:
        print reaction

        newmet_array = [0] * len(model.metabolites)
        met_count = 0
        for i in range(len(model.metabolites)):
            newmet_array[met_count] = string.replace(model.metabolites[i].id, " ", "_")
            met_count = met_count + 1

        #####ALGORITHM################
        # creates an empty dict to store DM_rxns as keys and the p_solution.f values
        normal_maxmet_dict = {}

        i=0
        #for i in range(80):

        #  switch toggle
        switch = True

        # this part creates a demand reaction for each metabolite
        temp_DM = 'DM_' + str(newmet_array[i])

        # this nested for loop tries to catch if a demand reaction already exists in the model
        for ii in model.reactions:
            if temp_DM == str(ii):
                # this part sets the newly created demand as the objective
                my_objective = model.reactions.get_by_id(str(temp_DM))
                model.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model.optimize(solver='gurobi')

                # check if p_solution is above a tolerance to check if we want to run loopless or not
                if p_solution < 10**-6:
                    normal_maxmet_dict[str(temp_DM)] = p_solution.f

                else:
                    solution = construct_loopless_model(model).optimize(solver='gurobi')
                    normal_maxmet_dict[str(temp_DM)] = solution.f

                # a toggle to see if the reaction is to be deleted or not
                switch = False

        if switch is True:
            temp_DM = Reaction(str(temp_DM))
            newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
            temp_DM.add_metabolites({newmet_array[i]: -1})
            model.add_reaction(temp_DM)

            # this part sets the newly created demand as the objective
            my_objective = model.reactions.get_by_id(str(temp_DM))
            model.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model.optimize(solver='gurobi')

            # check if p_solution is above a tolerance to check if we want to run loopless or not
            if p_solution < 10**-6:
                normal_maxmet_dict[str(temp_DM)] = p_solution.f

            else:
                solution = construct_loopless_model(model).optimize(solver='gurobi')
                normal_maxmet_dict[str(temp_DM)] = solution.f

            # this deletes the demand at the end of each iteration
            model.reactions.get_by_id(str(temp_DM)).delete()
            
        
        # working copy!
        # This creates an array of modified met names since I'm getting errors accessing model.metabolites[i].id in some of the code
        newmet_array = [0] * len(model_KO.metabolites)
        met_count = 0
        for i in range(len(model_KO.metabolites)):
            newmet_array[met_count] = string.replace(model_KO.metabolites[i].id, " ", "_")
            met_count = met_count + 1


        # creates an empty dict to store DM_rxns as keys and the p_solution.f values
        KO_maxmet_dict = {}

        i=0
        #for i in range(80):

        # switch toggle
        switch = True

        # this part creates a demand reaction for each metabolite
        temp_DM = 'DM_' + str(newmet_array[i])

        # this nested for loop tries to catch if a demand reaction already exists in the model
        for ii in model_KO.reactions:
            if temp_DM == str(ii):
                # this part sets the newly created demand as the objective
                my_objective = model_KO.reactions.get_by_id(str(temp_DM))
                model_KO.change_objective(my_objective)

                # this part runs FBA on the new demand
                p_solution = model_KO.optimize(solver='gurobi')

                # this stores the demand flux into a dictionary where its demand rxn is the key
                KO_maxmet_dict[str(temp_DM)] = p_solution.f

                # a toggle to see if the reaction is to be deleted or not
                switch = False

        if switch is True:
            temp_DM = Reaction(str(temp_DM))
            newmet_array[i] = model_KO.metabolites.get_by_id(model_KO.metabolites[i].id)
            temp_DM.add_metabolites({newmet_array[i]: -1})
            model_KO.add_reaction(temp_DM)

            # this part sets the newly created demand as the objective
            my_objective = model_KO.reactions.get_by_id(str(temp_DM))
            model_KO.change_objective(my_objective)

            # this part runs FBA on the new demand
            p_solution = model_KO.optimize(solver='gurobi')

            # this stores the demand flux into a dictionary where its demand rxn is the key
            KO_maxmet_dict[str(temp_DM)] = p_solution.f

            # this deletes the demand at the end of each iteration
            model_KO.reactions.get_by_id(str(temp_DM)).delete()


ARGSS


In [83]:
normal_maxmet_dict

{'DM_10fthf5glu_c': -0.0}

In [85]:
KO_maxmet_dict

{'DM_10fthf5glu_c': None}

In [71]:
model.objective

{}

In [123]:
normal_maxmet_dict = {}
test_met = model.metabolites.get_by_id('thym_c')
reaction = 'ARGSS'
# create a copy of reference model so don't have to worry about things accidently adding up
model = model_healthy_red.copy()

###### CREATE KO MODEL ######
model_KO = model.copy()
for reaction in unique_rxns[i1]:
    model_KO.reactions.get_by_id(reaction).lower_bound = 0
    model_KO.reactions.get_by_id(reaction).upper_bound = 0

# temporary string 
temp_DM = 'DM_' + str(test_met)
#creating the demand
temp_DM = Reaction(str(temp_DM))
#newmet_array[i] = model.metabolites.get_by_id(model.metabolites[i].id)
temp_DM.add_metabolites({test_met: -1})
model.add_reaction(temp_DM)

#set it as the objective
my_objective = model.reactions.get_by_id(str(temp_DM))
model.change_objective(my_objective)

#run FBA
p_solution = model.optimize(solver='gurobi')

## check if p_solution is above a tolerance to check if we want to run loopless or not
if p_solution < 10**-6:
    normal_maxmet_dict[str(temp_DM)] = p_solution.f

else:
    solution = construct_loopless_model(model).optimize(solver='gurobi')
    normal_maxmet_dict[str(temp_DM)] = solution.f

In [230]:
normal_maxmet_dict = {}
#test_met = model.metabolites.get_by_id('ura_e')
reaction = 'ARGSS'
# create a copy of reference model so don't have to worry about things accidently adding up
model = model_healthy_red.copy()

In [231]:
temp_DM = 'DM_ura_e'
temp_DM = Reaction(str(temp_DM))
temp_DM.add_metabolites({model.metabolites.get_by_id('ura_e'):-1})
temp_DM.reaction

'ura_e --> '

In [232]:
print model.objective
print model.optimize().f

{<Reaction biomass_NCI60 at 0x17d5a048>: 1}
0.0177


In [233]:
model.add_reaction(temp_DM)

In [234]:
print model.objective
print model.optimize().f

{<Reaction biomass_NCI60 at 0x17d5a048>: 1}
0.0177


In [235]:
my_objective = model.reactions.get_by_id(str(temp_DM))
model.change_objective(my_objective)

In [236]:
print model.objective
print model.optimize().f

{<Reaction DM_ura_e at 0x17d5ae10>: 1}
0.0


In [237]:
model_KO = model.copy()
model_KO.reactions.get_by_id('ARGSS').lower_bound = 0
model_KO.reactions.get_by_id('ARGSS').upper_bound = 0

In [238]:
print model_KO.objective
print model_KO.optimize()

{<Reaction DM_ura_e at 0x13d48438>: 1}
<Solution 'infeasible' at 0x17d5ae48>


In [221]:
model_KO_wARG = model_KO.copy()

In [222]:
model_KO_wARG.reactions.get_by_id('EX_arg__L_e').lower_bound = -0.0068

In [223]:
print model_KO_wARG.objective
print model_KO_wARG.optimize()

{<Reaction DM_thymd_c at 0x126fd1d0>: 1}
<Solution 'infeasible' at 0x1932fa20>


In [224]:
model_healthy_red.reactions[0].lower_bound

0.0

In [225]:
p_sol.x_dict['ARGSS']

0.004530956000000001

In [226]:
amp_c = model.metabolites.get_by_id('amp_c')
DM_amp_c = Reaction('DM_amp_c')
DM_amp_c.name = DM_amp_c
DM_amp_c.add_metabolites({amp_c: -1})
model.add_reaction(DM_amp_c)

argsuc_c = model.metabolites.get_by_id('argsuc_c')
DM_argsuc_c = Reaction('DM_argsuc_c')
DM_argsuc_c.name = DM_argsuc_c
DM_argsuc_c.add_metabolites({argsuc_c: -1})
model.add_reaction(DM_argsuc_c)

h_c = model.metabolites.get_by_id('h_c')
DM_h_c = Reaction('DM_h_c')
DM_h_c.name = DM_h_c
DM_h_c.add_metabolites({h_c: -1})
model.add_reaction(DM_h_c)

ppi_c = model.metabolites.get_by_id('ppi_c')
DM_ppi_c = Reaction('DM_ppi_c')
DM_ppi_c.name = DM_ppi_c
DM_ppi_c.add_metabolites({ppi_c: -1})
model.add_reaction(DM_ppi_c)

In [227]:
model.reactions.get_by_id('DM_amp_c').lower_bound = p_sol.x_dict['ARGSS']
model.reactions.get_by_id('DM_argsuc_c').lower_bound = p_sol.x_dict['ARGSS']
model.reactions.get_by_id('DM_h_c').lower_bound = p_sol.x_dict['ARGSS']
model.reactions.get_by_id('DM_ppi_c').lower_bound = p_sol.x_dict['ARGSS']

In [228]:
print model.objective
print model.optimize()

{<Reaction DM_thymd_c at 0x139dc860>: 1}
<Solution 'infeasible' at 0x1932f5c0>
